In [ ]:
from prophet import Prophet
import itertools
import numpy as np
import pandas as pd
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

data = pd.read_csv('data.csv')


param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here
mse = []
mae = []

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params, interval_width=0.9, daily_seasonality=True).fit(data)  # Fit model with given params
    df_cv = cross_validation(m, initial='100 days', period='60 days', horizon = '30 days')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])
    mse.append(df_p["mse"].values[0])
    mae.append(df_p["mae"].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
tuning_results['mse'] = mse
tuning_results['mae'] = mae
print(tuning_results)



In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)

In [ ]:
print(tuning_results)
#['rmse'].min()


In [ ]:
print(all_params)

In [ ]:
print(tuning_results[['rmse','mse','mae']].min().min())

In [ ]:
combined_row_value = []

for ind in tuning_results.index:
    sum_val = tuning_results['rmse'][ind] + tuning_results['mse'][ind] + tuning_results['mae'][ind]
    combined_row_value.append(dict(index=ind, sum_val=sum_val))

In [ ]:
print(combined_row_value)

In [ ]:
print(min(combined_row_value, key=lambda x:x['sum_val']))


In [ ]:
print(tuning_results.loc[32])